# Advanced Modeling for BTC Daily Features

Plan :
1. Implémenter des versions avancées des algos vus en cours (ElasticNet, GradientBoosting/HistGB, RandomForest/ExtraTrees).
2. Définir le plan apprentissage/test : choix des données, split chrono, contrôle de l'overfitting.
3. Analyser et critiquer les résultats avec MAE/RMSE/MAPE.
4. Combiner plusieurs algos (stacking/average) pour la décision.
5. Choisir un algo hors cours (réseau de neurones MLP) avec référence.
6. Expliquer l'algo et justifier le choix (référence scientifique).
7. Implémenter, évaluer et comparer avec les résultats précédents et le baseline.


## 0. Imports & Helpers
- Chargement des librairies
- Fonctions utilitaires (RMSE/MAPE, split chrono, évaluation)


In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor

pd.set_option('display.width', 200)

def rmse(y_true, y_pred):
    return float(mean_squared_error(y_true, y_pred) ** 0.5)

def mape(y_true, y_pred):
    pct = np.abs((y_true - y_pred) / y_true)
    pct = pd.Series(pct).replace([np.inf, -np.inf], np.nan).dropna()
    return float(pct.mean() * 100)

def chrono_split(X, y, train_ratio=0.7, val_ratio=0.15):
    n = len(X)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    return (X.iloc[:train_end], y.iloc[:train_end],
            X.iloc[train_end:val_end], y.iloc[train_end:val_end],
            X.iloc[val_end:], y.iloc[val_end:])

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return {
        'model': name,
        'MAE': mean_absolute_error(y_test, preds),
        'RMSE': rmse(y_test, preds),
        'MAPE%': mape(y_test, preds),
    }


### Explication (Imports & helpers)
- Fonctions utilitaires partagées : RMSE(Root Mean Squared Error)/MAPE (Mean Absolute Percentage Error)  pour les métriques, split chronologique pour éviter la fuite temporelle, `evaluate_model` pour entraîner + scorer.
- Les pipelines scaler+modèle assurent une standardisation cohérente quand nécessaire (linéaire, MLP).


## 1. Données et cible
- Chargement `btc_features_daily.csv`
- Tri chronologique
- Cible : close du lendemain (`target_close_next`)


In [ ]:

df = pd.read_csv(Path('btc_features_daily.csv'), parse_dates=['timestamp']).sort_values('timestamp')
df['target_close_next'] = df['close'].shift(-1)
df = df.iloc[:-1]

corr = (
    df.drop(columns=['timestamp'])
      .corr()['target_close_next']
      .drop('target_close_next')
      .abs()
      .sort_values(ascending=False)
)
print('Top 10 corrélations (absolues):')
print(corr.head(10))







NameError: name 'df' is not defined

In [ ]:
data_path = Path('btc_features_daily.csv')
df = pd.read_csv(data_path, parse_dates=['timestamp']).sort_values('timestamp').reset_index(drop=True)
print(f"Loaded {len(df):,} rows, {len(df.columns)} columns")
print('Missing ratio (top 5):')
print(df.isna().mean().head())

# Cible = close J+1
df['target_close_next'] = df['close'].shift(-1)
df = df.iloc[:-1]
feature_cols = [c for c in df.columns if c not in ['timestamp', 'target_close_next']]
X = df[feature_cols]
y = df['target_close_next']

X_train, y_train, X_val, y_val, X_test, y_test = chrono_split(X, y)
print(f"Splits -> train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")


### Explication (Données & cible)
- On charge `btc_features_daily.csv`, on trie par date, pas de valeurs manquantes sur les principales colonnes.
- Cible = close du lendemain (`target_close_next`) : formulation régressive, sensible à l'autocorrélation.
- Split 70/15/15 chronologique pour reproduire un scénario temps-réel.


## 2. Corrélations et contrôle de l'overfitting
- Corrélations très élevées entre niveaux de prix (indique redondance)
- Plan pour limiter l'overfit :
  - Standardisation pour modèles linéaires/MLP
  - Hyperparamètres contraints (profondeur, min_samples_leaf)
  - Split chrono (pas de fuite temporelle)


In [ ]:
corr = (
    df.drop(columns=['timestamp'])
      .corr()['target_close_next']
      .drop('target_close_next')
      .abs()
      .sort_values(ascending=False)
)
print('Top 8 corrélations absolues avec la cible:')
print(corr.head(8))


### Explication (Corrélations & overfitting)
- Les corrélations >0.99 entre prix (close/adj/typ_price) et la cible indiquent une forte redondance.
- Risque : les modèles complexes apprennent surtout à recopier le prix actuel → difficile de généraliser.
- Contre-mesures : régularisation, profondeur limitée, validation temporelle stricte.


## 3. Baseline
Baseline persistance : prédire que le close de demain = close d’aujourd’hui.


In [ ]:
baseline_preds = X_test['close']
baseline_res = {
    'model': 'persistence_close',
    'MAE': mean_absolute_error(y_test, baseline_preds),
    'RMSE': rmse(y_test, baseline_preds),
    'MAPE%': mape(y_test, baseline_preds),
}
baseline_res


### Explication (Baseline)
- Cette baseline prédit simplement que le close de demain = close d'aujourd'hui.
- Si ses MAE/RMSE/MAPE sont très bas, cela signifie que la série est fortement autocorrélée et difficile à battre.
- Tout modèle avancé doit faire mieux (MAE/RMSE plus petits, MAPE plus faible) pour être intéressant.


## 4. Modèles avancés (versions cours)
- ElasticNet (L1+L2) pour gérer colinéarité et sélection douce.
- Ridge (référence linéaire régularisée).
- GradientBoostingRegressor et HistGradientBoostingRegressor (plus robuste, bins).
- RandomForestRegressor et ExtraTreesRegressor (arbres bagging/variance réduite).


In [ ]:
models = [
    ('elasticnet', make_pipeline(StandardScaler(), ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000))),
    ('ridge', make_pipeline(StandardScaler(), Ridge(alpha=5.0))),
    ('gboost', GradientBoostingRegressor(random_state=0, n_estimators=400, learning_rate=0.02, max_depth=3)),
    ('hist_gb', HistGradientBoostingRegressor(random_state=0, max_depth=4, learning_rate=0.05, max_iter=500)),
    ('rf', RandomForestRegressor(n_estimators=400, random_state=0, n_jobs=-1, max_depth=6, min_samples_leaf=3)),
    ('extra_trees', ExtraTreesRegressor(n_estimators=400, random_state=0, n_jobs=-1, max_depth=6, min_samples_leaf=3)),
]

results = [baseline_res]
for name, model in models:
    results.append(evaluate_model(name, model, X_train, y_train, X_test, y_test))

pd.DataFrame(results).sort_values('RMSE')


### Explication (Baseline)
- Cette baseline prédit simplement que le close de demain = close d'aujourd'hui.
- Si ses MAE/RMSE/MAPE sont très bas, cela signifie que la série est fortement autocorrélée et difficile à battre.
- Tout modèle avancé doit faire mieux (MAE/RMSE plus petits, MAPE plus faible) pour être intéressant.


### Explication (Modèles avancés)
- **ElasticNet/Ridge** : testent si une régularisation linéaire peut extraire un léger signal malgré la colinéarité.
- **GB/HistGB** : capturent des non-linéarités; l'apprentissage lent (petit learning_rate) limite l'overfit.
- **RF/ExtraTrees** : bagging d'arbres, ici contraints (max_depth/min_samples_leaf) pour réduire l'overfit.
- Interprétation : comparez leurs métriques au baseline. Si elles sont plus grandes, ils n'apportent pas de gain; s'ils sont plus petites, ils surpassent la persistance.


## 5. Ensemble learning
On combine plusieurs modèles pour lisser leurs erreurs : moyenne simple et StackingRegressor (ridge meta-learner).


In [ ]:
# Fit base models
fitted = {}
for name, model in models:
    model.fit(X_train, y_train)
    fitted[name] = model

# Moyenne simple des prédictions des 4 meilleurs modèles (ElasticNet, Ridge, RF, HistGB)
selected = ['elasticnet','ridge','rf','hist_gb']
preds_avg = np.mean([fitted[n].predict(X_test) for n in selected], axis=0)
ensemble_avg = {
    'model': 'avg_elastic_ridge_rf_histgb',
    'MAE': mean_absolute_error(y_test, preds_avg),
    'RMSE': rmse(y_test, preds_avg),
    'MAPE%': mape(y_test, preds_avg),
}

# Stacking
estimators = [(n, fitted[n]) for n in selected]
stack = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=1.0), passthrough=False)
stack.fit(X_train, y_train)
preds_stack = stack.predict(X_test)
ensemble_stack = {
    'model': 'stack_elastic_ridge_rf_histgb',
    'MAE': mean_absolute_error(y_test, preds_stack),
    'RMSE': rmse(y_test, preds_stack),
    'MAPE%': mape(y_test, preds_stack),
}

ensemble_results = [ensemble_avg, ensemble_stack]
pd.DataFrame(results + ensemble_results).sort_values('RMSE')


### Explication (Ensemble learning)
- **Moyenne simple** : lisse les erreurs de modèles complémentaires; efficace si les erreurs ne sont pas corrélées.
- **Stacking** : méta-modèle (Ridge) apprend à pondérer les prédictions; peut capturer des interactions entre modèles.
- Lecture : si l'ensemble baisse MAE/RMSE/MAPE vs chaque modèle individuel et vs baseline, il apporte de la robustesse.


## 6. Modèle hors cours : MLP Regressor (réseau de neurones feed-forward)
Référence : Zhang, G. P. (2003), "Time series forecasting using a hybrid ARIMA and neural network model", *Neurocomputing*. Les MLP capturent des relations non linéaires; ici on l’emploie sur les features existantes.


In [ ]:
mlp = make_pipeline(
    StandardScaler(),
    MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=500, random_state=0, early_stopping=True)
)

mlp_res = evaluate_model('mlp_regressor', mlp, X_train, y_train, X_test, y_test)

pd.DataFrame(results + ensemble_results + [mlp_res]).sort_values('RMSE')


### Explication (MLP hors cours)
- MLP Regressor (réseau de neurones feed-forward) peut modéliser des relations non linéaires complexes.
- Early stopping réduit l'overfit; le scaling est crucial.
- Interprétation : comparez ses métriques à la baseline et aux autres. Si l'erreur est plus élevée, les features actuelles ne suffisent pas; si elle baisse, cela suggère un signal non linéaire exploité.


## 7. Analyse et comparaison
- Comparez chaque modèle au baseline persistance.
- Notez si l’ensemble (avg/stack) ou le MLP apportent un gain.
- Inspectez les métriques (MAE/RMSE/MAPE) et la robustesse.


In [ ]:
final_table = pd.DataFrame(results + ensemble_results + [mlp_res]).sort_values('RMSE')
final_table.reset_index(drop=True)


### Explication (Analyse finale)
- Triez par RMSE/MAE : plus petit = meilleur. La MAPE donne une lecture en %.
- Identifier le premier modèle qui bat clairement la baseline; sinon conclure que les features sont insuffisantes et qu'il faut changer la cible (retours/direction) ou ajouter des variables exogènes.
- Vérifier la stabilité : si un modèle surperforme mais avec très faible marge, valider en rolling split/val croisée temporelle.


### Conclusion rapide
- Si aucun modèle ne bat le baseline : les features n’apportent pas de signal supplémentaire; envisager des cibles en rendements/direction et des variables exogènes.
- Si un ensemble ou le MLP réduit la MAPE/MAE, poursuivre l’affinage (tuning hyperparam, rolling validation).
- Toujours valider en split temporel ou rolling pour éviter la fuite.
